In [1]:
from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/nacional.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180219 entries, 0 to 180218
Data columns (total 48 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   ID          180219 non-null  object 
 1   EDO         180219 non-null  int64  
 2   MES         180219 non-null  int64  
 3   ANIO        180219 non-null  int64  
 4   MPIO        180219 non-null  int64  
 5   HORA        180219 non-null  int64  
 6   MINUTOS     180219 non-null  int64  
 7   DIA         180219 non-null  int64  
 8   DIASEMANA   180219 non-null  int64  
 9   URBANA      180219 non-null  int64  
 10  SUBURBANA   180219 non-null  int64  
 11  TIPACCID    180219 non-null  int64  
 12  AUTOMOVIL   180219 non-null  int64  
 13  CAMPASAJ    180219 non-null  int64  
 14  MICROBUS    180219 non-null  int64  
 15  PASCAMION   180219 non-null  int64  
 16  OMNIBUS     180219 non-null  int64  
 17  TRANVIA     180219 non-null  int64  
 18  CAMIONETA   180219 non-null  int64  
 19  CA

<ipython-input-2-19b64c38e5df>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/nacional.csv')


In [3]:
accident_frequencies = df['TIPACCID'].value_counts()

# Display the frequencies
accident_frequencies


,count
TIPACCID,
1,125207
4,24419
10,16789
2,5905
11,2077
5,1871
12,1729
7,1245
6,676


In [4]:
# prompt: haz subsampling en "TIPACCID" (1) y en "CAUSAACCI" (1)

# Check for unique values in 'TIPACCID' and 'CAUSAACCI'
#print(df['TIPACCID'].unique())
#print(df['CAUSAACCI'].unique())

# If there are too many unique values, consider subsampling or other dimensionality reduction techniques.
# Example of subsampling 'TIPACCID' (replace with your preferred method):

# 1. Random Subsampling
#fraction = 0.2 # Adjust the fraction as needed
#df_subsampled = df.groupby('TIPACCID', group_keys=False).apply(lambda x: x.sample(frac=fraction))

# 2. Stratified Sampling (if you need proportional representation from each TIPACCID)
#from sklearn.model_selection import train_test_split
#df_subsampled = df.groupby('TIPACCID').apply(lambda x: x.sample(frac=0.2, random_state=1)).reset_index(drop=True)

# Display the subsampled data
#print(df_subsampled.head())

# Do the same for 'CAUSAACCI',  consider different sampling strategies depending on your needs


In [ ]:
import pandas as pd
from collections import Counter
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE # Import SMOTE
print("Distribución original:", Counter(df["TIPACCID"]))

# Separar features (X) y target (y)
X = df.drop(["TIPACCID", "CALLE1", "CALLE2"], axis=1)  # Todas las columnas excepto TIPACCID, CALL1, CALL2
y = df["TIPACCID"]  # Variable objetivo

# Configurar SMOTEENN
smote_enn = SMOTEENN(
    sampling_strategy="auto",  # Equilibra todas las clases
    random_state=42,           # Para reproducibilidad
    smote=SMOTE(k_neighbors=5) # Número de vecinos para SMOTE (ajustable)
)

# Aplicar el remuestreo
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# Convertir a DataFrame
accidentes_balanced = pd.DataFrame(X_resampled, columns=X.columns)
accidentes_balanced["TIPACCID"] = y_resampled

# Ver distribución final
print("Distribución después de SMOTEENN:", Counter(y_resampled))

Distribución original: Counter({1: 125207, 4: 24419, 10: 16789, 2: 5905, 11: 2077, 5: 1871, 12: 1729, 7: 1245, 6: 676, 3: 124, 9: 102, 8: 75})


In [ ]:
import pandas as pd
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Ver distribución original
print("Distribución original:", Counter(df["TIPACCID"]))

# Muestra del 30% para pruebas rápidas (opcional)
accidentes_sample = df.sample(frac=0.3, random_state=42)
X = accidentes_sample.drop(["TIPACCID", "CALLE1", "CALLE2"], axis=1)
y = accidentes_sample["TIPACCID"]

# --- ESTRATEGIA DE BALANCEO ---
# 1. Reducir "Colisión" (mayoritaria) a 50,000 (ajusta este número)
import pandas as pd
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Ver distribución original
print("Distribución original:", Counter(df["TIPACCID"]))

# Muestra del 50%
accidentes_sample = df.sample(frac=0.5, random_state=42)
X = accidentes_sample.drop(["TIPACCID", "CALLE1", "CALLE2"], axis=1)
y = accidentes_sample["TIPACCID"]

# --- ESTRATEGIA DE BALANCEO ---
under_strategy = {label: count for label, count in [(1, 50000)]}
over_strategy = {label: count for label, count in [(4, 25000), (10, 18000), (2, 7000), (11, 2000), (5, 2000), (12, 1900), (7, 1300), (6, 1200), (3, 1000), (9, 900), (8, 500)]
                  if label in y.unique()}
# Pipeline en 2 pasos
pipeline = Pipeline([
    ('under', RandomUnderSampler(sampling_strategy=under_strategy, random_state=42)),
    ('over', SMOTE(sampling_strategy=over_strategy, k_neighbors=3, random_state=42))
])

# Aplicar el balanceo
X_res, y_res = pipeline.fit_resample(X, y)

# Convertir a DataFrame
accidentes_balanced = pd.DataFrame(X_res, columns=X.columns)
accidentes_balanced["TIPACCID"] = y_res

# Ver distribución final
print("Distribución después del balanceo:", Counter(y_res))
pipeline = Pipeline([
    ('under', RandomUnderSampler(sampling_strategy=under_strategy, random_state=42)),
    ('over', SMOTE(sampling_strategy=over_strategy, k_neighbors=3, random_state=42))
])

# Aplicar el balanceo
X_res, y_res = pipeline.fit_resample(X, y)

# Convertir a DataFrame
accidentes_balanced = pd.DataFrame(X_res, columns=X.columns)
accidentes_balanced["TIPACCID"] = y_res

# Ver distribución final
print("Distribución después del balanceo:", Counter(y_res))

In [ ]:
df_subsampled = accidentes_balanced


In [ ]:
df_subsampled = accidentes_balanced  # Assuming accidentes_balanced is your original DataFrame

# Now, you can access the 'TIPACCID' column and get value counts:
df_subsampled_frequencies = df_subsampled['TIPACCID'].value_counts()

# Display the frequencies
print(df_subsampled_frequencies)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import folium
from sklearn.cluster import KMeans

# Re-assign df_subsampled to the original DataFrame containing the necessary columns
df_subsampled = accidentes_balanced  # Assuming accidentes_balanced is the original DataFrame

# 2. Extraer columnas necesarias
X = df_subsampled[['LATITUD', 'LONGITUD', 'TIPACCID']].values  # Use all data, not just head(100)

# 3. Definir número de clústeres (ajústalo según análisis)
K = 3

# 4. Aplicar K-Means
kmeans = KMeans(n_clusters=K, random_state=42)
df_subsampled['cluster'] = kmeans.fit_predict(X)  # Asignar cluster a cada punto
centroids = kmeans.cluster_centers_  # Coordenadas de centroides

# 5. Visualizar los resultados en un mapa interactivo
m = folium.Map(location=[df_subsampled['LATITUD'].mean(), df_subsampled['LONGITUD'].mean()], zoom_start=10)

# Definir colores para cada tipo de accidente
# Include all unique TIPACCID values as keys in accident_colors
unique_accident_types = df_subsampled['CAUSAACCI'].unique()
accident_colors = {acc_type: plt.cm.get_cmap('tab10')(i)  # Use a colormap to get distinct colors
                   for i, acc_type in enumerate(unique_accident_types)}

for _, row in df_subsampled.iterrows():
    # Use .get() to handle missing keys, defaulting to gray
    color = accident_colors.get(row['CAUSAACCI'], 'gray')
    folium.CircleMarker(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=4,
        color=color,  # Use the retrieved color or gray
        fill=True,
        fill_color=color,  # Use the retrieved color or gray
        fill_opacity=0.6
    ).add_to(m)

# Agregar centroides de clusters
for lat, lon, acc_type in centroids:
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="black", icon="info-sign"),
        popup=f"Centroide - Tipo {int(acc_type)}"
    ).add_to(m)

# Guardar mapa y abrir en navegador
m.save("mapa_accidentes.html")



In [ ]:
# prompt: ver con matplotlib la agrupacion

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Itera sobre los clusters
for cluster in range(K):
    # Use df_subsampled instead of df to access the 'cluster' column
    cluster_data = df_subsampled[df_subsampled['cluster'] == cluster]
    plt.scatter(cluster_data['LONGITUD'], cluster_data['LATITUD'], label=f'Cluster {cluster}')

# Plotea los centroides
plt.scatter(centroids[:, 1], centroids[:, 0], marker='*', s=200, c='black', label='Centroides')  # Nota: el índice [:,1] y [:,0] para LONGITUD y LATITUD respectivamente

plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.title('Agrupación de Accidentes con K-Means')
plt.legend()
plt.show()

In [ ]:

# Option 1: Silhouette Score per feature
from sklearn.metrics import silhouette_score

# Get the column names used for clustering
columnas_clustering = ['LATITUD', 'LONGITUD', 'TIPACCID']

feature_importances = {}
for col in columnas_clustering: # Iterate through column names
    kmeans = KMeans(n_clusters=5, random_state=0)
    kmeans.fit(df_subsampled[[col]])  # Fit KMeans using only one feature, selecting from df_subsampled
    score = silhouette_score(df_subsampled[[col]], kmeans.labels_)
    feature_importances[col] = score

sorted_features = dict(sorted(feature_importances.items(), key=lambda item: item[1], reverse=True))

print("variables mas importantes:")
for feature, score in sorted_features.items():
    print(f"{feature}: {score}")




In [ ]:
# prompt: crea un df para cada cluster

# Create a dictionary to store DataFrames for each cluster
cluster_dfs = {}

# Iterate through unique cluster labels
for cluster_label in df_subsampled['cluster'].unique():
    # Create a DataFrame for the current cluster
    cluster_dfs[cluster_label] = df_subsampled[df_subsampled['cluster'] == cluster_label]

# Access individual cluster DataFrames using their labels (e.g., cluster_dfs[0], cluster_dfs[1], etc.)
# Example: Print the first 5 rows of the DataFrame for cluster 0
print(cluster_dfs[0].head())

# You can now work with each cluster_dfs[i] DataFrame individually
# Example: Print some statistics for each cluster
for cluster_label, cluster_df in cluster_dfs.items():
    print(f"\nCluster {cluster_label} Statistics:")
    print(cluster_df.describe())


In [ ]:
cluster_dfs[1].head()

In [ ]:
for i in range(3):
  TIP_frequencies = cluster_dfs[i]['TIPACCID'].value_counts()
  TIP_frequencies = (TIP_frequencies/TIP_frequencies.sum())*100
  print("Cluster: ",i, TIP_frequencies)

In [ ]:
for i in range(3):
  causa_frequencies = cluster_dfs[i]['CAUSAACCI'].value_counts()
  causa_frequencies = (causa_frequencies/causa_frequencies.sum())*100
  print("Cluster: ",i, causa_frequencies)

In [ ]:
for i in range(3):
    tipaccid_heridos = cluster_dfs[i].groupby('TIPACCID')['TOTHERIDOS'].sum()

    max_heridos_tipaccid = tipaccid_heridos.idxmax()
    max_heridos_count = tipaccid_heridos.max()
    total_heridos  = cluster_dfs[i]['TOTHERIDOS'].sum()

    print(f"Cluster {i}:")
    print(f"TIPACCID con mas heridos: {max_heridos_tipaccid}")
    print(f"Total Heridos por TIPACCID: {max_heridos_count}")
    print(f"Total Heridos en el cluster: {total_heridos}")
    print("-" * 20)


In [ ]:
for i in range(3):
    tipaccid_muertos = cluster_dfs[i].groupby('TIPACCID')['TOTMUERTOS'].sum()

    max_muertos_tipaccid = tipaccid_muertos.idxmax()
    max_muertos_count = tipaccid_muertos.max()
    total_muertos  = cluster_dfs[i]['TOTMUERTOS'].sum()

    print(f"Cluster {i}:")
    print(f"TIPACCID con mas muertos: {max_muertos_tipaccid}")
    print(f"Total moridos por TIPACCID: {max_muertos_count}")
    print(f"Total de muertos en el cluster: {total_muertos}")
    print("-" * 20)


In [ ]:
for i in range(3):
  total_accidentes_cluster_0 = len(cluster_dfs[i])
  print(f"El total de accidentes en df_cluster:{i} {total_accidentes_cluster_0}")
  total_accidentes_cluster_0_2 = len(cluster_dfs[i][cluster_dfs[i]['TIPACCID'] == 2])
  print(f"El total de accidentes para peatones en df_cluster:{i}  {total_accidentes_cluster_0_2}")
  total_accidentes_cluster_0_10 = len(cluster_dfs[i][cluster_dfs[i]['TIPACCID'] == 10])
  print(f"El total de accidentes para vehículos en df_cluster:{i}  {total_accidentes_cluster_0_10}")
  total_accidentes_cluster_0_11 = len(cluster_dfs[i][cluster_dfs[i]['TIPACCID'] == 11])
  print(f"El total de accidentes para ciclistas en df_cluster:{i}  {total_accidentes_cluster_0_11}")

In [ ]:
for i in range(3):
  frecuencias = cluster_dfs[i]['TIPACCID'].value_counts()
  porcentaje = (frecuencias/frecuencias.sum())*100
  print(porcentaje)
